# Usar um pipeline para ler e processar um conjunto de ficheiros e usar esse mesmo pipeline como 'conjunto de treino' #

In [117]:
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Activation
from tensorflow.keras.optimizers import SGD




plt.style.use('seaborn-poster')
%matplotlib inline

A api, tf.data.Dataset , permite criar um pipeline de leitura de ficheiros e seu eventual processamento, usando a função 'map':
https://www.tensorflow.org/guide/data
,
https://www.tensorflow.org/guide/data_performance
.

Podemos fornecer diretamente um tfdata.Dataset ao método fit é uma forma muito eficiente de efetuar o treino de um modelo keras: https://www.tensorflow.org/guide/keras/train_and_evaluate#training_evaluation_from_tfdata_datasets


No que se segue exemplificaremos a utilização desta api num conjunto de ficheiros do 2ª trabalho. O tf.data.Dataset permite criar 'batches' que podem sevir de inputs e labels a um modelo keras que faça a segmentação dos sons cardíacos.

In [118]:
dir = './data/trainAlunos/'
list_npy_files = glob.glob(dir + '*.npy')
fnames_dataset_train = tf.data.Dataset.from_tensor_slices(list_npy_files)

In [119]:
print(fnames_dataset_train)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [120]:
it = iter(fnames_dataset_train)
ex = next(it)
ex

<tf.Tensor: shape=(), dtype=string, numpy=b'./data/trainAlunos\\13918_AV_sigTarg.npy'>

In [121]:
dir = './data/validacaoAlunos/'
list_npy_files = glob.glob(dir + '*.npy')
fnames_dataset_validacao = tf.data.Dataset.from_tensor_slices(list_npy_files)

In [122]:
print(fnames_dataset_validacao)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [123]:
it = iter(fnames_dataset_validacao)
ex = next(it)
ex

<tf.Tensor: shape=(), dtype=string, numpy=b'./data/validacaoAlunos\\14998_PV_sigTarg.npy'>

Podemos aceder um a um aos elementos (ou batches de elementos) de um dataset

A partir deste dataset inicial, fnames_dataset, vamos criar um cujo conteudo é o resultado de ler cada um dos ficheiros npy.

Começamos por uma função auxiliar que vai ser usada pela função 'map' para ler o conteúdo de cada um dos ficheiros do fnames_dataset e criar um novo dataset com estes conteúdos.

In [124]:
def read_npy(fname):
    """fname should be a npy file"""
    fname = fname.decode()
    recData = np.load(fname)
    return recData.astype(np.float32)

In [175]:
dataset_raw_train = fnames_dataset_train.map(lambda x: tf.numpy_function(read_npy, [x], [tf.float32]))
dataset_raw_validacao = fnames_dataset_validacao.map(lambda x: tf.numpy_function(read_npy, [x], [tf.float32]))


In [176]:


##ISTO É APENAS PARA O FEEDFORWARD, NOS OUTROS PODEMOS DAR O SINAL COMPLETO

dataset_cuted_train = dataset_raw_train.map(lambda x: x[:,(tf.shape(x)[1]-2000):tf.shape(x)[1]])
dataset_cuted_validacao = dataset_raw_train.map(lambda x: x[:,(tf.shape(x)[1]-2000):tf.shape(x)[1]])


In [206]:
it3 = iter(dataset_cuted_train)
ex3 = next(it3)
ex3

<tf.Tensor: shape=(2, 2000), dtype=float32, numpy=
array([[-266., -348., -379., ...,  -78.,  130.,  235.],
       [   4.,    4.,    4., ...,    3.,    3.,    3.]], dtype=float32)>

Poderiamos também usar a função 'map' para separar inputs e targets e obter um dataset que produz um 'tuple', (input, target) 

O nosso dataset pode também produzir 'batches' em vez de exemplos individuais.  No nosso caso os exempos não tẽm todos o mesmo comprimento, assim, teremos de extender(por zeros??) os exemplos mais curtos de cada batch de modo a todos os exemplos do mesmo batch terem o mesmo comprimento.

    Poderiamos também usar a função map para aplicar a 'stft' aos inputs 

In [177]:
frame_length= 1000
frame_step = 10

def stft(x):
    return tf.signal.stft(
    x,
    frame_length,
    frame_step,
    fft_length=None,
    window_fn=tf.signal.hann_window,
    pad_end=False,
    name=None
)

In [178]:
#precisar nos outros pra guardar tamnaho do sinal

""" 
 """

' dataset_stft_train = dataset_cuted_train.map(lambda x: ((tf.shape(x)[1],tf.signal.stft(x[0], 256,128)),x[1]))\ndataset_stft_validacao = dataset_cuted_validacao.map(lambda x: ((tf.shape(x)[1],tf.signal.stft(x[0], 256,128)),x[1]))\n '

In [328]:
dataset2x_train = dataset_cuted_train.map(lambda x: (tf.math.abs(tf.signal.stft(signals = x[0], frame_length = 256, frame_step = 128)), x[1]))
dataset2x_validacao = dataset_cuted_validacao.map(lambda x: (tf.math.abs(tf.signal.stft(signals = x[0], frame_length = 256, frame_step = 128)), x[1]))

In [329]:
it3 = iter(dataset2x_train)
ex3 = next(it3)
ex3

(<tf.Tensor: shape=(14, 129), dtype=float32, numpy=
 array([[4.0799033e+03, 1.2272309e+04, 7.2162002e+03, ..., 1.3904767e+02,
         2.7887966e+01, 1.8205566e+01],
        [4.2699473e+03, 1.2871683e+04, 1.0095986e+04, ..., 4.6013741e+01,
         1.1352508e+02, 6.6249756e+01],
        [4.5610225e+03, 7.2023442e+03, 1.5412860e+04, ..., 4.9293308e+01,
         8.5428505e+01, 2.5179688e+01],
        ...,
        [6.9241626e+03, 8.8304102e+03, 2.1321293e+04, ..., 8.2463585e+01,
         6.7195831e+01, 4.9052734e+01],
        [1.7521475e+04, 5.6102633e+04, 7.5388734e+04, ..., 1.1511120e+01,
         7.8482269e+01, 9.6849609e+01],
        [1.3976577e+03, 9.1582391e+04, 2.6074452e+05, ..., 9.0046486e+01,
         9.5586662e+01, 3.4563965e+01]], dtype=float32)>,
 <tf.Tensor: shape=(2000,), dtype=float32, numpy=array([4., 4., 4., ..., 3., 3., 3.], dtype=float32)>)

In [330]:
""" #precisar nos outros

batch_size = 2
k=129
dataset_pb_train = dataset_stft_train.padded_batch(batch_size, padded_shapes=(([], tf.TensorShape([None, k])),[None,]))
dataset_pb_validacao = dataset_stft_validacao.padded_batch(batch_size, padded_shapes=(([], tf.TensorShape([None, k])),[None,])) """

' #precisar nos outros\n\nbatch_size = 2\nk=129\ndataset_pb_train = dataset_stft_train.padded_batch(batch_size, padded_shapes=(([], tf.TensorShape([None, k])),[None,]))\ndataset_pb_validacao = dataset_stft_validacao.padded_batch(batch_size, padded_shapes=(([], tf.TensorShape([None, k])),[None,])) '

In [331]:
batch_size = 10

dataset_pb_train = dataset2x_train.batch(batch_size)
dataset_pb_validacao = dataset2x_validacao.batch(batch_size)

In [332]:
it3 = iter(dataset_pb_train)
ex3 = next(it3)
ex3

(<tf.Tensor: shape=(10, 14, 129), dtype=float32, numpy=
 array([[[4.07990332e+03, 1.22723086e+04, 7.21620020e+03, ...,
          1.39047668e+02, 2.78879662e+01, 1.82055664e+01],
         [4.26994727e+03, 1.28716826e+04, 1.00959863e+04, ...,
          4.60137405e+01, 1.13525078e+02, 6.62497559e+01],
         [4.56102246e+03, 7.20234424e+03, 1.54128604e+04, ...,
          4.92933083e+01, 8.54285049e+01, 2.51796875e+01],
         ...,
         [6.92416260e+03, 8.83041016e+03, 2.13212930e+04, ...,
          8.24635849e+01, 6.71958313e+01, 4.90527344e+01],
         [1.75214746e+04, 5.61026328e+04, 7.53887344e+04, ...,
          1.15111198e+01, 7.84822693e+01, 9.68496094e+01],
         [1.39765771e+03, 9.15823906e+04, 2.60744516e+05, ...,
          9.00464859e+01, 9.55866623e+01, 3.45639648e+01]],
 
        [[5.17522021e+03, 7.37476709e+03, 6.08029883e+03, ...,
          4.44938354e+01, 5.00777893e+01, 4.51113281e+01],
         [4.12469336e+03, 3.54550220e+03, 4.83382568e+03, ...,
          

In [350]:
model = Sequential()
model.add(keras.Input(shape=(14,129)))
model.add(Flatten())
model.add(Dense(256,  activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(5*2000))
model.add(Activation(activation="softmax"))
model.add(Flatten())
model.add(Reshape((2000,5)))
#model.add(Flatten())

In [334]:
""" model = Sequential()
model.add(keras.Input(shape=(14,129)))
model.add(Conv2D(32, 5, strides=2, activation="relu"))
model.add(Conv2D(32, 3, activation="relu"))
model.add(MaxPooling2D(3))
model.add(Flatten())
model.add(Dense(250, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(2))
model.summary() """

' model = Sequential()\nmodel.add(keras.Input(shape=(14,129)))\nmodel.add(Conv2D(32, 5, strides=2, activation="relu"))\nmodel.add(Conv2D(32, 3, activation="relu"))\nmodel.add(MaxPooling2D(3))\nmodel.add(Flatten())\nmodel.add(Dense(250, activation="relu"))\nmodel.add(Dense(100, activation="relu"))\nmodel.add(Dense(2))\nmodel.summary() '

In [335]:
""" model = models.Sequential([
    layers.Input(shape=(14,129)),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
]) """

" model = models.Sequential([\n    layers.Input(shape=(14,129)),\n    # Downsample the input.\n    layers.Resizing(32, 32),\n    # Normalize.\n    norm_layer,\n    layers.Conv2D(32, 3, activation='relu'),\n    layers.Conv2D(64, 3, activation='relu'),\n    layers.MaxPooling2D(),\n    layers.Dropout(0.25),\n    layers.Flatten(),\n    layers.Dense(128, activation='relu'),\n    layers.Dropout(0.5),\n    layers.Dense(num_labels),\n]) "

In [337]:
o= model(ex3[0])
o.shape

TensorShape([10, 2000, 5])

In [351]:
model.summary()

Model: "sequential_92"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_82 (Flatten)        (None, 1806)              0         
                                                                 
 dense_268 (Dense)           (None, 256)               462592    
                                                                 
 dense_269 (Dense)           (None, 128)               32896     
                                                                 
 dense_270 (Dense)           (None, 10000)             1290000   
                                                                 
 activation_57 (Activation)  (None, 10000)             0         
                                                                 
 flatten_83 (Flatten)        (None, 10000)             0         
                                                                 
 reshape_65 (Reshape)        (None, 2000, 5)         

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [352]:
# train the model using SGD
EPOCHS = 10

print("[INFO] training network...")
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='sgd',
	metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
H = model.fit(dataset_pb_train,
    epochs=EPOCHS)

[INFO] training network...
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'Equal_2' defined at (most recent call last):
    File "c:\Users\ggpt9\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\ggpt9\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\ggpt9\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\ggpt9\AppData\Local\Programs\Python\Python37\lib\site-packages\traitlets\config\application.py", line 972, in launch_instance
      app.start()
    File "c:\Users\ggpt9\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\ggpt9\AppData\Local\Programs\Python\Python37\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asy

In [ ]:
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[1]
        batchS = tf.shape(x)[0]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        positions = tf.expand_dims(positions, axis = 0)
        positions = tf.repeat(positions, batchS, axis=0)
        return x + positions

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
inputA = keras.Input(shape=([]))
inputB = keras.Input(shape=([None, 129]))
embDim = 10

#processa o input b e faz o upsampling e corta ao tamanho do target
newEmb = layers.TimeDistributed(layers.Dense(embDim))(inputB)
inputsPosEmb = PositionEmbedding(maxlen=20, embed_dim=embDim)(newEmb)
transf1 = TransformerBlock(embed_dim=embDim,num_heads=3, ff_dim=8)(inputsPosEmb)
transf2 =TransformerBlock(embed_dim=embDim,num_heads=3, ff_dim=8)(transf1)
out = layers.TimeDistributed(layers.Dense(3))(transf2)
#TODO insert LSTM here?

transf_model = keras.Model(inputs=inputB, outputs=out)

NameError: name 'keras' is not defined

In [ ]:
print(list(dataset3)[0][0][1].shape)

(2, 207, 129)


In [ ]:
lr = 1.0e-4
opt = tf.keras.optimizers.Adam(learning_rate=lr)
transf_model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics = 'accuracy')
transf_model.fit(inputsData, targets, batch_size=4, epochs = 50)

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show() 

In [ ]:
o= model)

sinal com tamanho inicial, 